In [1]:
import importlib

import numpy as np

from network_models.soundsream_models_and_utils.mapping_down.ss_direct_downmapping import SS_Direct_Downmapping_Model
from network_models.soundsream_models_and_utils.encoder.ss_encoder_downmapping import EncoderDownmapping
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 8
models_dir = "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/downmapping/0_0216/"
epochs =200
save_every = 40
start_lr = 1e-4
gc.collect()

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings_version0_12_1.pkl", device=device)

model = SS_Direct_Downmapping_Model(dropout=0.2, output=1024, start_dim=512 * 175).to(device)

2023-02-16 19:06:15.919906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 19:06:16.584651: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-16 19:06:16.584709: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-16 19:06:16.584713: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0] / "network_models/soundstream_models_and_utils/encoder")
if module_path not in sys.path:
    sys.path.append(module_path)
import network_models.soundsream_models_and_utils.mapping_down.ss_driect_dm_trainer as sset
importlib.reload(sset)


enc_trainer = sset.SSDirectDMTrainer(batch_size=batch_size, num_epochs=epochs, model_path=models_dir, save_model_every=save_every, lr=start_lr, dataset=data_set, device=device, model=model)
gc.collect()
enc_trainer.train()

NameError: name 'BatchSamplerForDiffClasses' is not defined

In [3]:
np1 = data_set.encoded_dataset.encodedData[data_set.encoded_dataset.labelcolumn].to_numpy()

In [45]:
class CustomBatchSampler(object):
    def __init__(self, targets, num_samples=1):
        self.targets = targets
        self.num_samples = num_samples
        self.num_classes = len(np.unique(self.targets))

    def __iter__(self):
        for i in range(self.num_classes):
            idxs = np.where(self.targets == i)[0]
            np.random.shuffle(idxs)
            num_batches = len(idxs) // self.num_samples
            for j in range(num_batches):
                  idxs[j*self.num_samples:(j+1)*self.num_samples]

    def __len__(self):
        return len(self.targets) // self.num_samples

In [48]:
class CustomBatchSampler(object):
    def __init__(self, targets):
        self.targets = targets
        self.num_classes = len(np.unique(self.targets))

        # Find the indices of the samples for each class
        self.class_idxs = {}
        for i in range(self.num_classes):
            idxs = np.where(self.targets == i)[0]
            if len(idxs) > 0:
                self.class_idxs[i] = idxs

    def __iter__(self):
        while len(self.class_idxs) > 0:
            batch_idxs = []
            for i in self.class_idxs.keys():
                idxs = self.class_idxs[i][:1]
                batch_idxs.extend(idxs)
                self.class_idxs[i] = np.delete(self.class_idxs[i], np.arange(len(idxs)))
                if len(self.class_idxs[i]) == 0:
                    del self.class_idxs[i]
                if len(batch_idxs) == self.num_classes:
                    break
            yield batch_idxs

    def __len__(self):
        return sum([len(idxs) for idxs in self.class_idxs.values()])

In [57]:
from torch.utils.data import DataLoader

l = [np1[i][0] for i in range(len(np1))]


dl = DataLoader(data_set, batch_sampler=CustomBatchSampler(np1))


for batch, (X, z) in enumerate(dl):
    print(np.argmax(z, axis=1))
    #print(z)
    # if batch == 4:
    #     break



tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0, 1, 2, 3, 4, 5, 6])
tensor([0,

RuntimeError: dictionary changed size during iteration

tensor([1, 3, 5])


RuntimeError: dictionary changed size during iteration

In [26]:
np1

array([array([3]), array([4]), array([3]), ..., array([5]), array([5]),
       array([1])], dtype=object)